Introduction:

This document discusses a simple sentence generator program, which mimicks real life language models, as well as details regarding how the algorithm functions. 

This program functions by recieving a list of sentences, and is able to output new sentences with a given starting word.


If you are too impatient to reading the whole document, here are some outputs generated by this program:


Note:
Please keep in mind that this project does not use actual machine learning elements, rather it was merely inspired by the general idea. More information and limitations of the program can be found in the end of the documentation.

Note 2:
Since this is a Jupyter notebook, you should be able to execute these programs directly in this document, provided that you downloaded it and have properly set up Python. However, you need to run the function declarations first before you can run the provided examples.

Section 1: Interpretation

This section discusses how the program reads in info, and how it processes said inputs for "learning".

In [1]:
#This function breaks down sentences into digestable chunks for the program to analyze

PUNCTS = (".", "?", "!") #Given list of currently accepted punctuations

def processSentence(sentence):
    chunks = []
    nextChunk = ""

    for i in sentence:
        if i in PUNCTS:
            chunks.append(nextChunk)
            nextChunk = ""

            chunks.append(i)
        elif i == " ":
            if len(nextChunk) > 0:
                chunks.append(nextChunk)
                nextChunk = ""
        else:
            nextChunk += i
    
    return chunks

Below is an example of the executed program:

In [8]:
sentence = "The quick brown fox jumps over the lazy dog."

print(processSentence(sentence))
#['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']


Section 2: Training

This section discusses how the program conform itself to the processed input, in preparation for sentence generating. This is the second level of preprocessing before the program can actually create its own outputs.

In [11]:
trainedMatrix = {
    #"SampleWord" : [[NextWords], [NextProbs]]
}

def train(trainingSentences, targetMatrix):
    for sen in trainingSentences:
        procSen = processSentence(sen)

        for i in range(len(procSen)):
            chk = procSen[i]
            prevChk = procSen[i-1] if i > 0 else None


            if not chk in PUNCTS and not chk in targetMatrix.keys():
                targetMatrix[chk] = [[], []]
            
            if prevChk is not None:

                if chk in targetMatrix[prevChk][0]:
                    updatedWrdList = targetMatrix[prevChk][0]
                    updatedProbList = targetMatrix[prevChk][1]

                    chkIndex = updatedWrdList.index(chk)
                    updatedProbList[chkIndex] += 1

                    targetMatrix.update({prevChk : [updatedWrdList, updatedProbList]})
                else:
                    updatedWrdList = targetMatrix[prevChk][0]
                    updatedProbList = targetMatrix[prevChk][1]

                    updatedWrdList.append(chk)
                    updatedProbList.append(1)

                    targetMatrix.update({prevChk : [updatedWrdList, updatedProbList]})

Below is an example of the executed program:

In [15]:
#Training sentences, each letter represents a word
testSentences = (
    "a b.",
    "b c.",
    "a c."
)

trainedMatrix = {
    #"SampleWord" : [[NextWords], [NextProbs]]
}

train(testSentences, trainedMatrix)

print(trainedMatrix)
#{'a': [['b', 'c'], [1, 1]], 'b': [['.', 'c'], [1, 1]], 'c': [['.'], [2]]}

#Here is a more visual representation of the created Markov Chain:
#   a b c .
# a 0 1 1 0 (both 'b' and 'c' appeared once after 'a' in all of the sentences)
# b 0 0 1 1
# c 0 0 0 2 (Note that the period appeared after 'c' twice, hence the 2 here)
# . - - - - (ending punctuations are a special case, as the program terminates everytime these symbols were generated)


{'a': [['b', 'c'], [1, 1]], 'b': [['.', 'c'], [1, 1]], 'c': [['.'], [2]]}


Section 3: Generation

This section touches on how the program uses all the preprocessed data to actually create its outputs.

The fundamental idea of most large problems are to divide it into smaller, more approachable tasks. Language models are no exception to this. As such, before considering how to generate a sentence, it may be helpful to consider how to generate the next word.

With the given processed data from the previous sections, we now know what words could be next, as well as their frequencies in the original training data. Therefore, our program can use this to its advantage to choose a random possible word, with the probability being decided by the frequencies of appearance.

Imagine a simple number range from 0 to 1. We can divide this range into different, smaller chunks of various sizes (for example: 0.5, 0.3, and 0.2). If we generate a random number between 0 and 1, this number would be in one of these chunks. Larger sized chunks means a higher chance that the random number would fall into its range. If we take this concept and map each chunk to a corresponding word, we have now created a simple way of picking a random word with varied probabilities.

Below are the two functions responsible for this task:

In [20]:
import random

#These functions create scaled chunks based on the given information, and then select a random chunk, see more in the documentation

#Helper function, this confines the probability range between 0 and 1
def probabilityGradient(probs):
    total = sum(probs)
    return [i/total for i in probs]

#This function returns the index of the randomly chosen section
def selectValue(probs):
    gradient = probabilityGradient(probs)
    target = random.random()
    total = 0
    result = 0

    for bound in gradient:
        total += bound

        if target <= total:
            return result
        result += 1

    #technically it would never reach this point but just return the last chunk in case it didn't pick anything prior
    return len(gradient) - 1

After being able to predict the next word, now the program can simply generate a full sentence through recursion, using its previous output as the next input.

In the case that the previous word has never be encountered before (if it was not part of the training data), the program would simply return an error. More information are in the documentation below regarding domains and limitations.

In [19]:
MAX_SENT_LIM = 50 #Hard stop maximum sentence length to prevent possible infinite recursion

def generate(startWrd, matrix):
    sent = [startWrd]

    if not startWrd in matrix.keys():
        print("Starting word hasn't been learnt before")
        return None

    def generateRec(prevWrd):
        if len(sent) >= MAX_SENT_LIM:
            return False
        else:
            nxtWords = matrix[prevWrd][0]
            nextProbs = matrix[prevWrd][1]

            nxtWord = nxtWords[selectValue(nextProbs)]
            sent.append(nxtWord)

            if nxtWord in PUNCTS:
                return True
            else:
                return generateRec(nxtWord)

    outcome = generateRec(startWrd)

    if outcome:
        print("Success, returning current result")
        return(sent)
    else:
        print("Exceeded maximum word limit, returning current result")
        return(sent)

Section 4: Results

This section provides example input and outputs of the program, while giving a deeper analysis on said inputs and outputs.

In [25]:
MAX_SENT_LIM = 50
PUNCTS = (".", "?", "!")

TRAIN_SENT = ( #generously provided by ChatGPT 3.5
    "The cat is sleeping on the mat.",
    "I enjoy reading books in my free time.",
    "She greeted me with a warm smile.",
    "The students are studying for their exams.",
    "They went for a walk in the park.",
    "The flowers in the garden are blooming beautifully.",
    "She is wearing a red dress to the party.",
    "They built a sandcastle on the beach.",
    "We visited the museum to see the art exhibition.",
    "The baby giggled at the funny sounds.",
    "He fixed the broken chair with a hammer.",
    "The birds are chirping in the trees.",
    "She danced gracefully on the stage.",
    "He painted a beautiful landscape on the canvas.",
    "The rain is pouring heavily outside.",
    "She gave a speech at the conference.",
    "He won a gold medal in the swimming competition.",
    "The children played happily in the playground.",
    "They built a snowman in the backyard.",
    "We took a family photo during the vacation.",
    "I solved the puzzle in record time.",
    "They went on a road trip across the country.",
    "We attended a music concert in the city.",
    "I practiced playing the piano every day.",
    "They enjoyed a sunset cruise on the lake.",
    "We volunteered at a local charity event."
)

#Actual code, run all of the function declarations in the previous function for this part to work.
train(TRAIN_SENT, trainedMatrix)

#Select a random valid starting word, taken from the training sentences
START_WORDS = ("I", "He", "She", "They", "The")
startWrd = START_WORDS[int(random.random() * len(START_WORDS))]

#This will provide a random result, have fun
print(generate(startWrd, trainedMatrix))

#The result is a list of the words, the official version of the program provides a helper function that turns it back into one string, but it is not included here.


Success, returning current result
['They', 'built', 'a', 'snowman', 'in', 'my', 'free', 'time', '.']


#Section 5: Implications and Limitations

This section discusses more about the general idea as well as possible future features that can greatly improve this very basic program.

*1: Domain of Input Sentences*

Currently, this program only accept very simple sentences, with only one independent clause. There is also a small given list of accepted ending punctuations (period, question mark, exclamation mark), and no commas are currently accepted as part of a valid training sentence. Despite this, the basis of multi-clause sentences are already avaliable, and may be updated in the near future.

*2: Sentence "Comprehension"*

It is obvious that this program generates rather nonsensical outputs, due to the recursive method only considering the previous word only. Larger, more advanced language models takes into account much more prior information.

*3: Word Choice*

Due to resource and personal limitations, this program simply picks words based on pure randomness, and the probability is only affected by appearance frequencies. In addition, if the training sentences are too unique from one another (sharing little words in common), there would be less opportunities for the program to creatively branch off. As such it'll just create one of the original training sentences, which is unfortunate and boring.

However theoretically, it may be possible to adjust and skew the probabilities using concepts from linear algebra. For example, a word can be weighted more heavily from another word to favor the former in generation. This can be achieved through matrix multiplications, and could be incredibly useful to control the feel of the result, such as making it sound more positive or negative in general. Unfortunately, more development into biases and weights would be needed for the generation of a more nuanced and clear narrative, and I'm not sure if I have the time or knowledge to pull it off.

Conclusion:

Overall, although this program obviously have little practical purposes (due to its limitations as well as being unable to compete with real language models), it may act as an oversimplified example for curious beginners who are looking into the field of artificial intelligence. At least, I had a fun time making this, and I hope that this document has been entertaining to read as well as being somewhat educational and insightful.

Thank you for reading.